In [2]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [7]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [4]:
client.create_experiment(name="my-cool-experiment")

'1'

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)


In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: fd3053fd69a44367b7a0281fbe3bbd9c, rmse: 6.3441
run id: 1feab41d0a0d4136b4e565cedcbb2b48, rmse: 6.4056
run id: 9635d930b1ab4d048e08198b8eebae12, rmse: 6.4199
run id: 738ae262029d4bd1ae7675a52e05cdff, rmse: 6.4541
run id: 3f868b596fc6480eacc1096e80519465, rmse: 6.4829


In [10]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [11]:
run_id = "fd3053fd69a44367b7a0281fbe3bbd9c"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Successfully registered model 'nyc-taxi-regressor'.
2022/06/05 20:22:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 1
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1654440738926, current_stage='None', description=None, last_updated_timestamp=1654440738926, name='nyc-taxi-regressor', run_id='fd3053fd69a44367b7a0281fbe3bbd9c', run_link=None, source='./mlruns/2/fd3053fd69a44367b7a0281fbe3bbd9c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [12]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


In [13]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1654440738926, current_stage='Staging', description=None, last_updated_timestamp=1654440748966, name='nyc-taxi-regressor', run_id='fd3053fd69a44367b7a0281fbe3bbd9c', run_link=None, source='./mlruns/2/fd3053fd69a44367b7a0281fbe3bbd9c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [14]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1654440738926, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2022-06-05', last_updated_timestamp=1654440753505, name='nyc-taxi-regressor', run_id='fd3053fd69a44367b7a0281fbe3bbd9c', run_link=None, source='./mlruns/2/fd3053fd69a44367b7a0281fbe3bbd9c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [15]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [16]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [18]:

client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

MlflowException: The following failures occurred while downloading one or more artifacts from ./mlruns/2/fd3053fd69a44367b7a0281fbe3bbd9c/artifacts: {'preprocessor': "FileNotFoundError(2, 'No such file or directory')"}